<a href="https://colab.research.google.com/github/vaishnavi2038/chat-bot/blob/main/chatbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install tflearn
import tensorflow as tf
import numpy as np
import tflearn
import random 
import json 


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents (9).json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "location",\n         "patterns": ["What is your location?", "Where are 

In [ ]:
with open("intents.json") as json_data:
  intents=json.load(json_data)

In [ ]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [ ]:
words=[]
classes=[]
documents=[]
ignore=['?']              # to exclude some punctuation marks
# looping through each sentence 
for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    # toktnize
    w=nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w,intent["tag"]))
    if intent['tag'] not in classes:
      classes.append(intent["tag"])
        


In [ ]:
# performing stemming and into a lower case 
words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

#remove duplicate clases 
classes=sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classes")
print(len(words),"unique words",words)


31 documents
9 classes
57 unique words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [ ]:
# creating training data 
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
print(len(train_x))
print(len(train_y))

31
31


In [ ]:
#reseting underlying graph data 
tf.compat.v1.reset_default_graph

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=346, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 1383  | total loss: 1.96358 | time: 0.039s
| Adam | epoch: 346 | loss: 1.96358 - acc: 0.9041 -- iter: 24/31
Training Step: 1384  | total loss: 1.77748 | time: 0.056s
| Adam | epoch: 346 | loss: 1.77748 - acc: 0.9137 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
import pickle 
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data","wb"))

In [ ]:
# restoring all data structures 
data=pickle.load(open("training_data","rb"))
words=data["words"]
classes=data["classes"]
train_x=data["train_x"]
train_y=data["train_y"]


In [ ]:

with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [ ]:
# taking user sentences 
def clean_up_sentence(sentence):
  #tokenizing 
  sentence_words=nltk.word_tokenize(sentence)
  #stemming
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence,words,show_details=False):
  # tokenizing 
  sentence_words=clean_up_sentence(sentence)
  # creating bag of words 
  bag=[0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if s==w:
        bag[i]=1
        if show_details:
          print("found in bag: %s" %w)
  return (np.array(bag))



In [ ]:
ERROR_THRESHOLD=0.25
def classify(sentence):
  #generate probabilties
  results=model.predict([bow(sentence,words)])[0]
  #filtering the predictions below threshold 
  results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  #sorting the predictions 
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  return return_list

In [ ]:
context={}
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)
        

In [ ]:
classify('What are you hours of operation?')

[('hours', 0.81510395)]

In [ ]:
response('What are you hours of operation?')

We're open every day 9am-9pm


In [ ]:
response("where are you?")

We are situated at the intersection of London Alley and Bridge Avenue


In [ ]:
response("What is menu for today?")

Our speciality for today is Chicken Tikka


In [ ]:
response("Hi there!")

Hi there, how can I help?
